# **<center><span style='font-family:Georgia'><span style="color:white;font-weight:bold"> <span style='background:#1A2540'> 🚢 Titanic Survival Prediction </span>**

<img src="https://imgur.com/oLUbTMG.gif">

**This is my first notebook on kaggle. As this is my first notebook I keep on updating the notebook after learning new and better presentation of codes and plots.**
**This is most basic dataset among the begineers of machine learning entusiast. There are better and robust approaches out there but I love to learn anything a bit simpler and understandable so here is my notebook lets get started.**

**The task is to predict survival of passangers with 0 = "Not Survived" and 1 = "Survived"**

# **<center><span style='font-family:Georgia'><span style="color:white;font-weight:bold"> <span style='background:#1A2540'> 📕Importing Libraries </span>**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#split
from sklearn.model_selection import train_test_split

#score and Accuracy
from sklearn.metrics import precision_score, accuracy_score, confusion_matrix

#models
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier, Perceptron, PassiveAggressiveClassifier
from sklearn.svm import SVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.neighbors import NearestCentroid,KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,GradientBoostingClassifier
from xgboost import XGBClassifier

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Color Palettes
colors = ["#1F2642","#DF7979", "#B6B6B6"]
sns.palplot(sns.color_palette(colors))

**Theme:**

I love to pre select my color pallet for the visualization so it will look clean and consitent through out the vizualization.

# **<center><span style='font-family:Georgia'><span style="color:white;font-weight:bold"> <span style='background:#1A2540'> 💽Importing DataSet </span>**


In [ ]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")

train_data.head().style.set_caption("Train Dataframe").set_properties(**{'background-color': colors[2],
                                                                         'color':colors[0],'border': '1.5px  solid black'})

In [ ]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")

test_data.head().style.set_caption("Test Dataframe").set_properties(**{'background-color': colors[2],
                                                                         'color':colors[0],'border': '1.5px  solid black'})

The Dataset contains some nan values we will take a look.

# **<center><span style='font-family:Georgia'><span style="color:white;font-weight:bold"> <span style='background:#1A2540'>  🔍Exploratory Data Analysis </span>**

### <span style="color:#F65557;font-weight:bold"> Filling missing Values <span>

In [ ]:
train_data.isnull().sum()/len(train_data)* 100

In [ ]:
test_data.isnull().sum()/len(train_data)* 100

As we can see there are 3 columns with some missing values and imputing them is important for the model to predict Accurately.

* Cabin has more than 50% missing data so we will remove it from both data.

In [ ]:
#imputing the Embarked with mode
train_data['Embarked'].fillna(train_data['Embarked'].mode()[0], inplace = True)

#removing column Cabin with missing data more than 50%
train_data.drop(['Cabin'], axis=1, inplace=True)
test_data.drop(['Cabin'], axis=1, inplace=True)

#imputing the Age with meadian 
train_data['Age'].fillna(train_data['Age'].median(),inplace = True)
test_data['Age'].fillna(test_data['Age'].median(),inplace = True)

#impiting fare in test data with median
test_data['Fare'].fillna(test_data['Fare'].median(), inplace = True)

### <span style="color:#F65557;font-weight:bold"> Balance of Target column <span>

In [ ]:
sns.countplot(x=train_data['Survived'], palette = colors);

Balanced Data "Hurray"

In [ ]:
sns.countplot(x=train_data['Pclass'], palette = colors);

In [ ]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = round(sum(women)/len(women)* 100)

men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = round(sum(men)/len(men) * 100)

print(f"Women who survived: {rate_women}% ")
print(f"Men who survived: {rate_men}%")

# **<center><span style='font-family:Georgia'><span style="color:white;font-weight:bold"> <span style='background:#1A2540'>  🧮Model Selection and Predicting </span>**

In [ ]:
features = ["Pclass", "Sex", "SibSp", "Parch","Age"] # Feature Selection

X = pd.get_dummies(train_data[features])
y = train_data["Survived"] # Target Value

X_testing = pd.get_dummies(test_data[features].fillna(-1))

In [ ]:
#spliting the train test
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state = 42)
print(f" Xtrain:{X_train.shape}, ytrain:{y_train.shape}")

In [ ]:
#finding the best models with base parameters
models = [
    ('Logistic Regression', LogisticRegression(max_iter=1000)),
    ('Ridge', RidgeClassifier()),
    ('SGD Classifier', SGDClassifier(max_iter=1000, tol=1e-3)),
    ('Support Vector Classifier', SVC()),
    ('NuSVC', NuSVC()),
    ('Decision Tree', DecisionTreeClassifier()),
    ('Gaussian NB', GaussianNB()),
    ('Bernoulli NB', BernoulliNB()),
    ('Perc', Perceptron()),
    ('Nearest Centroid', NearestCentroid()),
    ('Random Forest Classifier', RandomForestClassifier()),
    ('Ada Boost Classifier', AdaBoostClassifier()),
    ('XGB Classifier', XGBClassifier(verbosity = 0)),
    ('Passive Aggressive', PassiveAggressiveClassifier())
]

results = dict()
for name, model in models:
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    
    
    score = accuracy_score(y_test, pred)
    cm = confusion_matrix(y_test,pred)
    precision = precision_score(y_test, pred) 
    results[name] = score 

##### I selected multiple model to see the baseline of the models and which baseline gives the good score before tuning the parameters

In [ ]:
#storing the results into dataframe
df_results = pd.DataFrame([results])
df_results = df_results.transpose()
df_results = df_results.rename(columns={0:'Score'}).sort_values(by='Score',ascending=False)

In [ ]:
df_results.style.set_properties(**{'background-color': colors[1],
                                    'color': colors[0],
                                    'border': '0.5px  solid black'})

In [ ]:
fig = plt.figure(figsize=(15,9))

ax = sns.barplot(data=df_results, 
                 y=df_results.index, 
                 x='Score',
                 saturation=0.8,
                 linewidth=0.1,
                 color = colors[0])


##### After looking checking the multiple models the best performing models are XGB and Random Forest Classifier

In [ ]:
model = RandomForestClassifier()
model.fit(X_train,y_train)
predict= model.predict(X_test)

### <span style="color:#F65557;font-weight:bold"> Accuracy <span>

In [ ]:
from sklearn.metrics import plot_roc_curve
A = plot_roc_curve(model, X_test, y_test);

In [ ]:
fig,ax = plt.subplots(figsize=(4,4))
ax= sns.heatmap(confusion_matrix(y_test, predict),
                annot = True,
                cbar= True,
                fmt='g',
                cmap = colors)
plt.xlabel('True Label')
plt.ylabel('Predicted Label')
plt.title('Confusion-Matrix')
plt.tight_layout()  
plt.show()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predict))

# **<center><span style='font-family:Georgia'><span style="color:white;font-weight:bold"> <span style='background:#1A2540'>  🛠️ Hyper Parameter Tuning </span>**

### <span style="color:#F65557;font-weight:bold">Tuning Random Forest Classifer with Optuna <span>

    
##### Using the Bayesian Optimization hyper parameter tuner (Optuna)

In [ ]:
import optuna

import sklearn.datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score


def objective(trial):
    x, y = X_train,y_train

    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    max_depth = trial.suggest_int("max_depth", 2, 100, log=True)
    n_estimators = trial.suggest_int("n_estimators", 1,1000)    
    min_samples_split = trial.suggest_int("min_samples_split",1,5)
    min_samples_leaf = trial.suggest_int("min_samples_leaf",1,5)
    

    rf = sklearn.ensemble.RandomForestClassifier(criterion =criterion,
            max_depth=max_depth, 
            n_estimators=n_estimators
        )

    score = cross_val_score(rf, x, y, n_jobs=-1, cv=3)
    accuracy = score.mean()
    return accuracy


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=101)

In [ ]:
trial = study.best_trial
print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

Using the best parameters in the model

In [ ]:
rfc_params = {'criterion': 'gini', 'max_depth': 5, 'n_estimators': 196, 'min_samples_split': 4, 'min_samples_leaf': 1}

model = RandomForestClassifier(**rfc_params)
model.fit(X_train,y_train)
prediction= model.predict(X_test)

In [ ]:
sub_pre = model.predict(X_testing)

# **<center><span style='font-family:Georgia'><span style="color:white;font-weight:bold"> <span style='background:#1A2540'> <span style='background:#1A2540'> 📦 Submission</span>**

In [ ]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': sub_pre})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

In [ ]:
output.head().style.set_caption("Submission Dataframe").set_properties(**{'background-color': colors[2],
                                                                         'color':colors[0],'border': '1.5px  solid black'})

# **<center><span style='font-family:Georgia'><span style="color:white;font-weight:bold"> <span style='background:#1A2540'> <span style='background:#1A2540'>THE END</span>**

<h2><center> <span style="font-family:Georgia"> <span style="color:white;font-weight:bold"> <span style="background:#1A2540">✌️ If you like my notebook and found it usefull please do upvote</span></span></span></center></h2>